In [5]:
'''
Created on 31 July

@author: Ray

Next_Step: we can choose which subset of 
Reference: https://hcmy.gitbooks.io/ycimpute/content/shi-yong-knn/yuan-li.html
'''

from ycimpute.imputer.knnimput import KNN # inputation library
import pandas as pd
import numpy as np
import time
import os
import multiprocessing as mp # for speeding up some process
import sys
sys.path.append('../py_model')
from utils import init_logging
import logging 


In [6]:
#---------------------------------------------
# Step1: Loading base_features
#---------------------------------------------

# setting

log_dir = '../log_knn_inputation'
init_logging(log_dir)

df = pd.read_hdf('../features/base_featurs.h5','base_featurs')

copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index']
df.drop(no_need_to_inpute, axis = 1, inplace = True)

logging.info('shape of base_featurs : {}'.format(df.shape) )
#---------------------------------------------
# Step2: split dataframe into multiple dataframe to avoid memory problem
#---------------------------------------------

small_df_ls = []
# shuffle
df = df.sample(frac = 1.0, replace = False) # without replacement

num_split = 25
previous_step = None
for i, step in enumerate(np.arange(0, df.shape[0], step = int(df.shape[0] / num_split))):
    # for memory problem, we cannot feed all the data points into algorithm, which depends on row and n_features
    if i == 0:
        pass
    elif i == 1:
        small_df_ls.append(df[0: step])
        previous_step = step
    elif i == (num_split):
        small_df_ls.append(df[previous_step: ])
        break
    else:
        small_df_ls.append(df[previous_step: step])
        previous_step = step

# unit testing
logging.info('no mistake on splitting based on shape' if pd.concat(small_df_ls, axis = 0).shape[0] == len(df) else 'oops, sth wrong in splitting')
logging.info('no mistakg on splltting based on length' if len(small_df_ls) == num_split  else "oops, sth wrong in splitting")


shape of base_featurs : (356251, 280)
shape of base_featurs : (356251, 280)
no mistake on splitting based on shape
no mistake on splitting based on shape
no mistakg on splltting based on length
no mistakg on splltting based on length


In [7]:

#---------------------------------------------
# Step3: imputatiing based on imputation method u chossed
#---------------------------------------------

def imputation(k):
	for drop_targtet in [False]: 
		#-------------------
		# imputating
		#------------------
		small_df_filled_ls = []
		for small_df in small_df_ls:
			# preprocessing for algorithm
			if drop_targtet == True:
				# avoidnig using TARGET to impute
				name = 'wo_target'
				X_missing = small_df.copy()
				X_missing.drop(['TARGET'], axis =1, inplace = True)
			else:
				name = 'w_target'
				X_missing = small_df.copy()
			# core algorithm: input should be array
			X_filled = KNN(k = k, verbose = True).complete(X_missing.values)
			small_df_filled_ls.append(pd.DataFrame(X_filled, columns = X_missing.columns))

		logging.info('k : {}'.format(k))
		logging.info('case: {}'.format(name))
		#-------------------
		# output
		#-------------------
		df_filled = pd.concat(small_df_filled_ls, axis = 0)
		df_filled.SK_ID_CURR = df_filled.SK_ID_CURR.astype(int)
		logging.info('there will be no bugging in merge' if df_filled.SK_ID_CURR.nunique() == df.SK_ID_CURR.nunique() else "opps")

		#
		if drop_targtet == True: 
			pass
		else:
			df_filled.drop(['TARGET'], axis = 1, inplace = True)

		df_filled = pd.merge(df_filled, copy_for_the_following_merge, on = 'SK_ID_CURR', how = 'left')
		logging.info('final_shape : {}'.format(df_filled.shape))
		#-------------------
		# save
		#-------------------
		output_path = '../features/filled_by_knn'
		if not os.path.isdir(output_path):
			os.mkdir(output_path)

		df_filled.to_hdf(
			os.path.join(output_path, 'base_featurs_filled_knn_{}_{}.h5'.format(k, name)), 
			'base_featurs_filled_knn_{}_{}.h5'.format(k, name))

def multi(k):
	'''
	It's for using multi preprosessing to speed up the process.

	parameters:
	---------------------
	T: 5, 10, 15, 20
	'''
	imputation(k)



In [8]:
##################################################
# Main
##################################################

s = time.time()

mp_pool = mp.Pool(15) # 4 == len of the below list
mp_pool.map(multi, [k for k in np.arange(25, 100, step = 5)]) 

e = time.time()

logging.info('{} secs'.format(e-s))


/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/site-packages/ycimpute/unsupervised/knn/normalized_distance.py:52: RuntimeWarning: invalid value encountered in subtract
  np.subtract(X, x.reshape((1, n_cols)), out=diffs)
/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/site-packages/ycimpute/unsupervised/knn/normalized_distance.py:52: RuntimeWarning: invalid value encountered in subtract
  np.subtract(X, x.reshape((1, n_cols)), out=diffs)
/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/site-packages/ycimpute/unsupervised/knn/normalized_distance.py:52: RuntimeWarning: invalid value encountered in subtract
  np.subtract(X, x.reshape((1, n_cols)), out=diffs)
/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/site-packages/ycimpute/unsupervised/knn/normalized_distance.py:52: RuntimeWarning: invalid value encountered in subtract
  np.subtract(X, x.reshape((1, n_cols)), out=diffs)
/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/site-packages/ycimpute/unsupervised/knn/normaliz

Imputing row 1/14250 with 116 missing, elapsed time: 674.523
Imputing row 1/14250 with 116 missing, elapsed time: 675.229
Imputing row 101/14250 with 116 missing, elapsed time: 675.341
Imputing row 101/14250 with 116 missing, elapsed time: 676.041
Imputing row 201/14250 with 13 missing, elapsed time: 676.078
Imputing row 301/14250 with 34 missing, elapsed time: 676.750
Imputing row 201/14250 with 13 missing, elapsed time: 676.806
Imputing row 401/14250 with 98 missing, elapsed time: 677.455
Imputing row 301/14250 with 34 missing, elapsed time: 677.526
Imputing row 1/14250 with 116 missing, elapsed time: 677.831
Imputing row 1/14250 with 116 missing, elapsed time: 678.043
Imputing row 501/14250 with 111 missing, elapsed time: 678.175
Imputing row 401/14250 with 98 missing, elapsed time: 678.235
Imputing row 101/14250 with 116 missing, elapsed time: 678.647
Imputing row 101/14250 with 116 missing, elapsed time: 678.866
Imputing row 601/14250 with 36 missing, elapsed time: 678.917
Imputin

Imputing row 1801/14250 with 7 missing, elapsed time: 692.439
Imputing row 2001/14250 with 104 missing, elapsed time: 692.499
Imputing row 2101/14250 with 46 missing, elapsed time: 692.657
Imputing row 101/14250 with 116 missing, elapsed time: 693.089
Imputing row 2501/14250 with 99 missing, elapsed time: 693.118
Imputing row 2001/14250 with 104 missing, elapsed time: 693.137
Imputing row 2601/14250 with 50 missing, elapsed time: 693.143
Imputing row 2101/14250 with 46 missing, elapsed time: 693.210
Imputing row 1901/14250 with 44 missing, elapsed time: 693.213
Imputing row 2201/14250 with 133 missing, elapsed time: 693.347
Imputing row 201/14250 with 13 missing, elapsed time: 693.808
Imputing row 2101/14250 with 46 missing, elapsed time: 693.839
Imputing row 2601/14250 with 50 missing, elapsed time: 693.842
Imputing row 2701/14250 with 0 missing, elapsed time: 693.896
Imputing row 2201/14250 with 133 missing, elapsed time: 693.914
Imputing row 2301/14250 with 39 missing, elapsed time:

Imputing row 3901/14250 with 104 missing, elapsed time: 706.050
Imputing row 3701/14250 with 37 missing, elapsed time: 706.024
Imputing row 4301/14250 with 10 missing, elapsed time: 706.132
Imputing row 4301/14250 with 10 missing, elapsed time: 706.135
Imputing row 3901/14250 with 104 missing, elapsed time: 706.534
Imputing row 4101/14250 with 43 missing, elapsed time: 706.715
Imputing row 3801/14250 with 93 missing, elapsed time: 706.656
Imputing row 4001/14250 with 7 missing, elapsed time: 706.771
Imputing row 2001/14250 with 104 missing, elapsed time: 706.814
Imputing row 4401/14250 with 96 missing, elapsed time: 706.847
Imputing row 4401/14250 with 96 missing, elapsed time: 706.910
Imputing row 4001/14250 with 7 missing, elapsed time: 707.234
Imputing row 4201/14250 with 51 missing, elapsed time: 707.455
Imputing row 4101/14250 with 43 missing, elapsed time: 707.482
Imputing row 2101/14250 with 46 missing, elapsed time: 707.542
Imputing row 3901/14250 with 104 missing, elapsed time

Imputing row 5801/14250 with 132 missing, elapsed time: 719.325
Imputing row 6001/14250 with 43 missing, elapsed time: 719.343
Imputing row 6201/14250 with 124 missing, elapsed time: 719.573
Imputing row 5601/14250 with 74 missing, elapsed time: 719.536
Imputing row 5801/14250 with 132 missing, elapsed time: 719.751
Imputing row 3801/14250 with 93 missing, elapsed time: 719.753
Imputing row 6001/14250 with 43 missing, elapsed time: 719.970
Imputing row 5901/14250 with 92 missing, elapsed time: 720.091
Imputing row 6101/14250 with 102 missing, elapsed time: 720.092
Imputing row 5701/14250 with 41 missing, elapsed time: 720.246
Imputing row 6301/14250 with 52 missing, elapsed time: 720.342
Imputing row 5901/14250 with 92 missing, elapsed time: 720.518
Imputing row 3901/14250 with 104 missing, elapsed time: 720.628
Imputing row 6101/14250 with 102 missing, elapsed time: 720.645
Imputing row 6001/14250 with 43 missing, elapsed time: 720.785
Imputing row 6201/14250 with 124 missing, elapsed

...

there will be no bugging in merge
there will be no bugging in merge


Imputing row 10201/14251 with 32 missing, elapsed time: 744.402
Imputing row 10301/14251 with 97 missing, elapsed time: 745.181


final_shape : (356251, 280)
final_shape : (356251, 280)
/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'base_featurs_filled_knn_90_w_target.h5'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


Imputing row 10401/14251 with 49 missing, elapsed time: 745.997
Imputing row 10501/14251 with 18 missing, elapsed time: 746.740
Imputing row 10601/14251 with 130 missing, elapsed time: 747.624
Imputing row 10701/14251 with 36 missing, elapsed time: 748.321
Imputing row 10801/14251 with 35 missing, elapsed time: 749.022
Imputing row 10901/14251 with 64 missing, elapsed time: 749.614
Imputing row 11001/14251 with 24 missing, elapsed time: 750.198
Imputing row 11101/14251 with 35 missing, elapsed time: 750.856
Imputing row 11201/14251 with 7 missing, elapsed time: 751.568
Imputing row 11301/14251 with 74 missing, elapsed time: 752.265
Imputing row 11401/14251 with 69 missing, elapsed time: 753.047
Imputing row 11501/14251 with 51 missing, elapsed time: 753.895
Imputing row 11601/14251 with 126 missing, elapsed time: 754.545
Imputing row 11701/14251 with 27 missing, elapsed time: 755.300
Imputing row 11801/14251 with 39 missing, elapsed time: 755.938
Imputing row 11901/14251 with 87 missin

k : 75
k : 75
case: w_target
case: w_target
there will be no bugging in merge
there will be no bugging in merge
final_shape : (356251, 280)
final_shape : (356251, 280)
/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'base_featurs_filled_knn_75_w_target.h5'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
20489.722091436386 secs
20489.722091436386 secs


In [4]:
[k for k in np.arange(25, 100, step = 5)]

[25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]